Implementation of the Inception-V4 model. Although this is a quite heavy one in terms of computational load and parameters, and it might be redundant to use this one for simple tasks, I have implemented it to better get used to the syntax of keras. Feel free to use in any need.

Find the article under this link: https://arxiv.org/pdf/1602.07261.pdf

In [68]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt


In [69]:
def batchNormalized_conv2d(input, numOfKernels,kernelSize,padding,strides=1):
  layer = keras.layers.Conv2D(numOfKernels,kernelSize,padding=padding,strides=strides,activation="relu")(input)
  layer = keras.layers.BatchNormalization()(layer)

  return layer

def stem(input):
  x = batchNormalized_conv2d(input,32,3,padding="valid",strides=2)
  x = batchNormalized_conv2d(x,32,3,padding="valid",strides=1)
  x = batchNormalized_conv2d(x,64,3,padding="same")

  x_1 = batchNormalized_conv2d(x,96,3,padding="valid",strides=2)
  x_2 = keras.layers.MaxPool2D(pool_size=3, strides=2, padding="valid")(x)
  
  x = keras.layers.Concatenate(axis = 3)([x_1,x_2])

  x_1 = batchNormalized_conv2d(x,64,1,padding="same")
  x_1 = batchNormalized_conv2d(x_1,64,kernelSize=(7,1),padding="same")
  x_1 = batchNormalized_conv2d(x_1,64,kernelSize=(1,7),padding="same")
  x_1 = batchNormalized_conv2d(x_1,96,3,padding="valid")

  x_2 = batchNormalized_conv2d(x,64,1,padding="same")
  x_2 = batchNormalized_conv2d(x_2,96,3,padding="valid")

  x = keras.layers.Concatenate(axis = 3)([x_1,x_2])

  x_1 = keras.layers.MaxPool2D(pool_size=2,strides=2, padding="valid")(x) # pool_size=2 also as default
  x_2 = batchNormalized_conv2d(x,192,3,padding="valid",strides=2)

  x = keras.layers.Concatenate(axis = 3)([x_1,x_2])

  return x

def inception_A(input):

  x_1 = batchNormalized_conv2d(input,64,1,padding="same")
  x_1 = batchNormalized_conv2d(x_1,96,3,padding="same")
  x_1 = batchNormalized_conv2d(x_1,96,3,padding="same")
  
  x_2 = batchNormalized_conv2d(input,64,1,padding="same")
  x_2 = batchNormalized_conv2d(x_2,96,3,padding="same")

  x_3 = batchNormalized_conv2d(input,96,1,padding="same")

  x_4 = keras.layers.AveragePooling2D()(input)
  x_4 = batchNormalized_conv2d(input,96,1,padding="same")

  x = keras.layers.Concatenate(axis = 3)([x_1,x_2,x_3,x_4])


  return x

def inception_B(input):
  
  x_1 = batchNormalized_conv2d(input, 192, 1,padding="same")
  x_1 = batchNormalized_conv2d(x_1,192,(1,7),padding="same")
  x_1 = batchNormalized_conv2d(x_1,224,(7,1),padding="same")
  x_1 = batchNormalized_conv2d(x_1,224,(1,7),padding="same")
  x_1 = batchNormalized_conv2d(x_1,256,(7,1),padding="same")

  x_2 = batchNormalized_conv2d(input, 192, 1, padding="same")
  x_2 = batchNormalized_conv2d(x_2,224,(1,7),padding="same")
  x_2 = batchNormalized_conv2d(x_2,256,(1,7),padding="same")

  x_3 = batchNormalized_conv2d(input,384,1,padding="same")

  x_4 = keras.layers.AveragePooling2D(pool_size=3,padding="same",strides=1)(input)
  x_4 = batchNormalized_conv2d(x_4,128,1,padding="same")

  x = keras.layers.Concatenate(axis = 3)([x_1,x_2,x_3,x_4]) 

  return x

def inception_C(input):

  x_1 = batchNormalized_conv2d(input,384,1,padding="same")
  x_1 = batchNormalized_conv2d(x_1,448,(1,3),padding="same")
  x_1 = batchNormalized_conv2d(x_1,512,(3,1),padding="same")
  x_1_1 = batchNormalized_conv2d(x_1,256,(1,3),padding="same")
  x_1_2 = batchNormalized_conv2d(x_1,256,(3,1),padding="same")

  x_1 = keras.layers.Concatenate(axis = 3)([x_1_1,x_1_2])

  x_2 = batchNormalized_conv2d(input,384,1,padding="same")
  x_2_1 = batchNormalized_conv2d(x_2,256,(3,1),padding="same")
  x_2_2 = batchNormalized_conv2d(x_2,256,(3,1),padding="same")

  x_2 = keras.layers.Concatenate(axis = 3)([x_2_1, x_2_2])

  x_3 = batchNormalized_conv2d(input,256,1,padding="same")

  x_4 = keras.layers.AveragePooling2D(3,strides=1,padding="same")(input)
  x_4 = batchNormalized_conv2d(x_4,256,1,padding="same")

  x = keras.layers.Concatenate(axis = 3)([x_1,x_2,x_3,x_4])

  return x

def reduction_A(input):

  x_1 = batchNormalized_conv2d(input,192,1,padding="same")
  x_1 = batchNormalized_conv2d(x_1,224,3,padding="same")
  x_1 = batchNormalized_conv2d(x_1,256,3,padding="valid", strides=2)

  x_2 = batchNormalized_conv2d(input,384,3,padding="valid", strides=2)

  x_3= keras.layers.MaxPool2D(3, strides = 2)(input)

  x = keras.layers.Concatenate(axis = 3)([x_1,x_2,x_3])
  
  return x

def reduction_B(input):

  x_1 = batchNormalized_conv2d(input,256,1,padding="same")
  x_1 = batchNormalized_conv2d(x_1,256,(1,7),padding="same")
  x_1 = batchNormalized_conv2d(x_1,320,(7,1),padding="same")
  x_1 = batchNormalized_conv2d(x_1,320,3,padding="valid", strides=2)

  x_2 = batchNormalized_conv2d(input,192,1,padding="same")
  x_2 = batchNormalized_conv2d(x_2,192,3,padding="valid", strides=2)

  x_3= keras.layers.MaxPool2D(3, strides = 2)(input)

  x = keras.layers.Concatenate(axis = 3)([x_1,x_2,x_3])

  return x

def inception_v4(inputShape):

  input = keras.layers.Input(shape=inputShape)

  x = stem(input)

  x = inception_A(x)
  x = inception_A(x)
  x = inception_A(x)
  x = inception_A(x)

  x = reduction_A(x)

  x = inception_B(x)
  x = inception_B(x)
  x = inception_B(x)
  x = inception_B(x)
  x = inception_B(x)
  x = inception_B(x)
  x = inception_B(x)

  x = reduction_B(x)

  x = inception_C(x)
  x = inception_C(x)
  x = inception_C(x)

  x = keras.layers.GlobalAveragePooling2D()(x)
  x = keras.layers.Dropout(0.2)(x)
  x = keras.layers.Dense(units = 1000, activation='softmax')(x) 

  model = keras.Model(inputs = input , outputs = x , name ='Inception-V4')

  return model

In [70]:
model = inception_v4((299,299,3))

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

model.summary()

Model: "Inception-V4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_23 (InputLayer)          [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1727 (Conv2D)           (None, 149, 149, 32  896         ['input_23[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_1727 (Batc  (None, 149, 149, 32  128        ['conv2d_1727[0][0]']            
 hNormalization)                )                                                      